Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [36]:
from random import choices, random, randint
import numpy as np
from copy import deepcopy

import lab9_lib

# GOAL: maximize fitness, minimize calls

In [37]:
fitness = lab9_lib.make_problem(5)
for n in range(20):
    ind = choices([0, 1], k=100)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

0110111010010001000010111011010000011111010011010011100001010011010111100101111111110010000000000000: 9.89%
0100011011101000110111101011010101111111001010011111010011111110011100111110011100000001001111101000: 26.70%
0101001110010110010000000110101110100100011101001100101001101110001111001101110010010110001100000001: 20.91%
1101011111001100011100011111011101000100011001000101011011011010110110111001110001001110100111011101: 24.79%
0110011111000001110000100100110101111000101111001100000100111000000001110001110111100001111010101000: 10.90%
1110101011000101111100011010101011001011110101110101011110011100000001110101100010011011111110011001: 13.67%
1011100100101001101010110001000101010101000000001010100101011111110010011001000100001110000111001111: 10.80%
1101000010001111010010110100011111011000010001001110000011001100001111111010011000000010111101101001: 20.91%
0010110001001101010100111010011110110111001100000101000000000010111110101001110111010100110011001001: 23.01%
00011001001000111101

Our code below

In [38]:
POPULATION_SIZE = 100
OFFSPRING_SIZE = 100
LOCI = 1000
BIT_FLIP_PROBABILITY = 0.20
NUM_GENERATION = 100


class Individual:
    def __init__(self):
        self.genotype = choices([0, 1], k=LOCI)
        self.fitness = float("-inf")

In [39]:
# Mutation / recombination or both
def parent_selection(
    population: list[Individual], tournament_size: int
) -> list[Individual]:
    # we also want to take the last best one.
    parents_idx = np.random.choice(
        range(len(population)), size=tournament_size - 1, replace=False
    )
    parents = [population[idx] for idx in parents_idx if idx != 0]
    return [population[0], max(parents, key=lambda i: i.fitness)]


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, LOCI - 1)
    new_ind = Individual()
    new_ind.genotype = ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    assert len(new_ind.genotype) == LOCI
    return new_ind


# This xover function returns a child whose genome
# is created proportionally to the fitenss of parents
def uniform_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    p1 = ind1.fitness / (ind1.fitness + ind2.fitness)
    gene = [
        np.random.choice([ind1.genotype[i], ind2.genotype[i]], p=[p1, 1 - p1])
        for i in range(LOCI)
    ]
    new_ind = Individual()
    new_ind.genotype = gene
    return new_ind


def mutate(parent: Individual) -> Individual:
    new_offspring = deepcopy(parent)
    for i in range(LOCI):
        if random() < BIT_FLIP_PROBABILITY:
            new_offspring.genotype[i] = int(not new_offspring.genotype[i])
    return new_offspring


def offspring_generation(
    parent1: Individual, parent2: Individual, mutation_probability: int
) -> Individual:
    if random() < mutation_probability:
        # mutation
        return mutate(parent1)
    else:
        # cross_over
        return uniform_cut_xover(parent1, parent2)


def ea() -> Individual:
    # starting pouplation of POPULATION_SIZE individuals
    population = [Individual() for _ in range(POPULATION_SIZE)]
    for p in population:
        p.fitness = fitness(p.genotype)
    old_best_fitness = 0
    best_fitness = population[0].fitness
    # for _ in range(NUM_GENERATION)
    gen = 0
    mutation_probability = 0.20
    tournament_size = 10
    while best_fitness != old_best_fitness or gen < NUM_GENERATION:
        old_best_fitness = best_fitness
        num_of_better_offspring = 0
        # best_old_one = best_one
        best_parents = parent_selection(population, tournament_size)
        for _ in range(OFFSPRING_SIZE):
            offspring = offspring_generation(
                best_parents[0], best_parents[1], mutation_probability
            )
            offspring.fitness = fitness(offspring.genotype)
            population.extend([offspring])
            # tracking the number of offspring better than the best old one.
            if offspring.fitness > old_best_fitness:
                num_of_better_offspring += 1

        population.sort(key=lambda i: i.fitness, reverse=True)
        # always keep the first POPULATION_SIZE best individuals
        population = population[:POPULATION_SIZE]

        # Self-adapting the values.

        if num_of_better_offspring >= 0.2 * OFFSPRING_SIZE:
            # If we are able to generate 20% of offsprings better than the father,
            # We should explore more.
            tournament_size += 1
            mutation_probability *= 1.1
        else:
            # We are probably around a good point, we should favour exploitation.
            tournament_size = tournament_size - 1 if tournament_size > 5 else 5
            mutation_probability /= 1.1

        best_fitness = population[0].fitness
        gen += 1

    return population[0]

In [40]:
# instance = [1, 2, 5, 10]
instance = [2]
for k in instance:
    fitness = lab9_lib.make_problem(k)
    print(f"Best individual fitness: {ea().fitness}")
    print(f"Fitness calls: {fitness.calls}")

Best individual fitness: 0.556
Fitness calls: 10100
